In [30]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pyreadr
import yaml
from os.path import join
from datetime import datetime
import json
import geopandas as gpd

%matplotlib inline

In [31]:
with open(join('..', '..', 'config.yml')) as f:
    config = yaml.safe_load(f)
    
pwd = config['pwd']
dpath = config['storage']

In [46]:
df = pd.read_csv(join(pwd, 'util_datasets', 'nyc-subway-ridership.csv'))
df.date = pd.to_datetime(df.date)
df = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.gtfs_longitude, df.gtfs_latitude))
df.geometry = df.geometry.set_crs("EPSG:4326")

zcta_df = gpd.read_file(join(pwd, 'util_datasets', 'NYC', 'NYC-zcta.json'))

In [47]:
merged = gpd.sjoin(df, zcta_df, how='left', op='within')

In [53]:
merged.groupby(['zcta', 'date']).sum()[['entries', 'exits']].reset_index().to_csv(join(pwd, 'util_datasets', 'nyc-subway-ridership-grouped.csv'))